In [4]:
import pandas as pd
import numpy as np

# 1. 시계열/통계 데이터 (df_time)
np.random.seed(42)
dates = pd.date_range('2023-01-01', periods=100, freq='D')
prices = np.random.lognormal(mean=2, sigma=1, size=100) * 1000 # 왜도가 있는 데이터 생성
sales = np.random.randint(10, 100, size=100).astype(float)

df_time = pd.DataFrame({'Date': dates, 'Price': prices, 'Sales': sales})

# 결측치 강제 생성 (보간법 연습용)
df_time.loc[10:20, 'Sales'] = np.nan 
df_time.loc[50:55, 'Sales'] = np.nan

# 2. 넓은 형태 데이터 (df_wide) - Melt 연습용
data_wide = {
    'Product': ['A', 'B', 'C'],
    '2023-Q1': [100, 150, 200],
    '2023-Q2': [110, 160, 210],
    '2023-Q3': [105, 155, 205],
    '2023-Q4': [120, 170, 220]
}
df = pd.DataFrame(data_wide)

print("데이터 생성 완료!")
print(df_time.head())
print(df.head())

데이터 생성 완료!
        Date         Price  Sales
0 2023-01-01  12142.529844   18.0
1 2023-01-02   6434.896126   71.0
2 2023-01-03  14121.359907   46.0
3 2023-01-04  33886.945659   60.0
4 2023-01-05   5846.520075   53.0
  Product  2023-Q1  2023-Q2  2023-Q3  2023-Q4
0       A      100      110      105      120
1       B      150      160      155      170
2       C      200      210      205      220


In [5]:
skew_orig = df_time['Price'].skew()
#print(skew_orig)

df_time['Price_Log'] = np.log1p(df_time['Price'])
skew_log = df_time['Price_Log'].skew()
result1 = abs(skew_orig - skew_log)
print(round(result1, 2))

2.07


In [6]:
df_time['Sales_fill'] = df_time['Sales'].interpolate(method='linear')
df_time['Prev_Sales'] = df_time['Sales_fill'].shift(1)
result2 = df_time['Sales_fill'].corr(df_time['Prev_Sales'])
print(round(result2, 2))
print(df_time.head())

0.25
        Date         Price  Sales  Price_Log  Sales_fill  Prev_Sales
0 2023-01-01  12142.529844   18.0   9.404552        18.0         NaN
1 2023-01-02   6434.896126   71.0   8.769646        71.0        18.0
2 2023-01-03  14121.359907   46.0   9.555515        46.0        71.0
3 2023-01-04  33886.945659   60.0  10.430815        60.0        46.0
4 2023-01-05   5846.520075   53.0   8.673773        53.0        60.0


In [7]:
#print(df.head())
df_melt = pd.melt(df, id_vars=['Product'], var_name='Quarter', value_name='Revenue')
#print(df_melt)

result3 = df_melt[df_melt['Quarter']=='2023-Q4']['Revenue'].sum()
print(round(result3))

510


In [8]:
min_val = df_time['Price'].min()
max_val = df_time['Price'].max()
df_time['Price_Scaled'] = (df_time['Price'] - min_val) / (max_val - min_val)
result4 = sum(df_time['Price_Scaled'] > 0.5)
print(result4)

8


📝 실전 연습 문제
문제 1. (조건 필터링 & 중앙값 대치) Price 컬럼의 결측치(NaN)를 **City별 Price의 중앙값(median)**으로 채우시오. 그 후, Payment가 'Card'인 데이터들의 Price 평균값을 구하시오. (단, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하시오.)

문제 2. (시계열 & Groupby) Date 컬럼에서 '요일'(Monday=0, ... Sunday=6) 정보를 추출하여 Weekday 컬럼을 만드시오. **'주말' (토요일, 일요일)**에 해당하는 데이터만 필터링한 후, 이 데이터들의 City별 Price **합계(sum)**를 구했을 때 가장 큰 값을 구하시오. (단, 정수로 출력하시오.)

문제 3. (데이터 구간화 & 빈도수) Price 컬럼 값(결측치 처리 완료된 상태 가정)을 기준으로 데이터를 3개의 구간으로 나눕니다.

0 이상 4000 미만: 'Low'

4000 이상 7000 미만: 'Mid'

7000 이상: 'High'

'Mid' 그룹에 속하면서 Product가 'A'인 데이터의 **개수(행의 수)**를 구하시오.

In [9]:
import pandas as pd
import numpy as np

np.random.seed(2024)

# 데이터 생성
dates = pd.date_range('2023-01-01', periods=500, freq='h')
cities = np.random.choice(['Seoul', 'Busan', 'Daegu', 'Incheon'], 500)
products = np.random.choice(['A', 'B', 'C'], 500)
prices = np.random.randint(1000, 10000, 500)
payments = np.random.choice(['Card', 'Cash', 'Pay'], 500)

df = pd.DataFrame({
    'Date': dates,
    'City': cities,
    'Product': products,
    'Price': prices,
    'Payment': payments
})

# 결측치 임의 생성 (Price 컬럼)
df.loc[10:20, 'Price'] = np.nan

# 데이터 확인
print(df.head())

                 Date     City Product   Price Payment
0 2023-01-01 00:00:00    Seoul       A  2725.0    Cash
1 2023-01-01 01:00:00    Daegu       B  9929.0    Card
2 2023-01-01 02:00:00    Seoul       C  3118.0    Card
3 2023-01-01 03:00:00    Seoul       C  2901.0    Card
4 2023-01-01 04:00:00  Incheon       A  2739.0    Cash


In [10]:
# 문제 1
fill_nan = df.groupby('City')['Price'].transform('median')

df['Price'] = df['Price'].fillna(fill_nan)
#print(df.info())
result = df[df['Payment']=='Card']['Price'].mean()
print(round(result, 2))

5426.29


In [11]:
# 문제 2
df['Weekday'] = df['Date'].dt.weekday
q1 = df['Weekday'].isin([5, 6])
result = df.groupby('City')['Price'].sum().sort_values(ascending=False)
print(result)
# loc, iloc, isin 쓰는법


City
Incheon    815941.0
Busan      647059.5
Daegu      641636.0
Seoul      599213.0
Name: Price, dtype: float64


In [12]:
def get_category(price):
    if price < 4000: return 'Low'
    elif price < 7000: return 'Mid'
    else: return 'High'

df['Category'] = df['Price'].apply(get_category)
result = len(df[(df['Category']=='Mid') & (df['Product']=='A')])
print(result)

50


📝 실전 연습 문제 (Large Data)
문제 1. (시계열 분해 & 집계) Date 컬럼을 이용하여 **2023년의 '월별(Month)' 총 범죄 건수(Violent + Theft + Traffic)**를 집계하시오. 그중 총 범죄 건수가 **가장 많았던 월(Month)**의 범죄 건수 합계를 구하시오. (단, 정수로 출력하시오.)

문제 2. (복합 연산 & Groupby 주의점) 전체 기간 동안 각 구(District)별로 데이터가 수집되었습니다. **'Zone_A'**의 연간 10만 명당 평균 강력 범죄(Violent) 발생 건수를 구하시오.

계산 로직: (Zone_A의 전체 Violent 합계 / 데이터 수집 년수) / Zone_A의 인구수 * 100,000

데이터 수집 년수(Years)는 5년(2020~2024)으로 계산합니다.

결과값은 소수점 이하는 버리고 **정수(int)**로 출력하시오.

문제 3. (이상치 탐색 & 조건부 평균) 전체 데이터 중 Traffic 범죄 건수가 상위 1%(quantile(0.99)) 이상인 날들을 **'Traffic_High'**로 정의합니다. 이 'Traffic_High' 그룹에 속하는 데이터들의 Violent 범죄 발생 건수의 평균을 구하시오. (단, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하시오.)

In [13]:
import pandas as pd
import numpy as np

# 랜덤 시드 고정 (정답 통일을 위해)
np.random.seed(2024)

# 1. 기본 설정
n_rows = 50000
districts = [f'Zone_{chr(i)}' for i in range(65, 85)] # Zone_A ~ Zone_T (20개)

# 2. 구별 인구수 매핑 (Zone_A는 30만, Zone_B는 35만... 랜덤 배정)
pop_map = {d: np.random.randint(200000, 800000) for d in districts}

# 3. 데이터 생성
df = pd.DataFrame({
    'Date': np.random.choice(pd.date_range('2020-01-01', '2024-12-31'), n_rows),
    'District': np.random.choice(districts, n_rows),
    'Violent': np.random.randint(0, 15, n_rows),  # 일일 발생 건수라 적게 설정
    'Theft': np.random.randint(0, 30, n_rows),
    'Traffic': np.random.randint(0, 50, n_rows)
})

# 4. 인구수 매핑 및 정렬
df['Population'] = df['District'].map(pop_map)
df = df.sort_values('Date').reset_index(drop=True)

print(f"데이터 생성 완료! 크기: {df.shape}")
print(df.head(10))

데이터 생성 완료! 크기: (50000, 6)
        Date District  Violent  Theft  Traffic  Population
0 2020-01-01   Zone_N        6     21       21      662347
1 2020-01-01   Zone_A       10      3       44      732104
2 2020-01-01   Zone_N        4      8       38      662347
3 2020-01-01   Zone_E       12      0       47      654593
4 2020-01-01   Zone_K       13      5       35      474799
5 2020-01-01   Zone_H        9      5       37      721696
6 2020-01-01   Zone_D        0      7       20      661246
7 2020-01-01   Zone_G       14      5       41      486794
8 2020-01-01   Zone_O       11     28       41      466331
9 2020-01-01   Zone_S        1     10        9      253094


In [14]:
# 문제 1
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month

df['Total'] = df['Violent'] + df['Theft'] + df['Traffic']
q1 = df[df['year']==2023]
result = q1.groupby('month')['Total'].sum().sort_values(ascending=False)
print(result.iloc[0])



41536


In [15]:
# 문제 2
q2 = df[df['District']=='Zone_A']
year_ratio = (q2['Violent'].sum() / 5) / q2['Population'].iloc[0] * 100000
print(int(year_ratio))

479


In [16]:
# 문제 3
var = df['Traffic'].quantile(0.99)
def cond(traffic):
    if traffic >= var: return 'Traffic_High'
    else: return 'NoNan'

df['Traffic_check'] = df['Traffic'].apply(cond)

result = df[df['Traffic_check']=='Traffic_High']['Violent'].mean()
print(round(result, 2))

6.94


📝 실전 연습 문제 (약점 보완 & 문자열)
문제 1. (문자열 추출 & 빈도 분석) Product 컬럼에는 대괄호 [ ... ] 안에 제품의 옵션 정보가 들어있습니다. (예: [Black]) 정규표현식을 사용하여 이 옵션 텍스트만 추출(괄호 제외)하여 Option이라는 새 컬럼을 만드세요. 가장 많이 판매된(행 개수 기준) Option의 이름을 출력하시오.

문제 2. (그룹별 결측치 처리 & 평균 계산) Price 컬럼의 결측치를 **Category별 Price의 평균값(mean)**으로 채우세요. (주의: 전체 평균이 아니라, Electronics는 Electronics끼리, Accessories는 Accessories끼리 평균을 구해서 채워야 합니다.) 결측치 처리가 완료된 후, Price의 **표준편차(std)**를 구하시오. (단, 소수점 이하는 버리고 **정수(int)**로 출력하시오.)

문제 3. (시계열 집계 & 이상치 범위 - 지난번 실수 복습) 일별 매출액(Daily_Sales)을 계산하려고 합니다. (매출액 = Price * Quantity, 결측치 처리된 Price 사용) 날짜(Date)별로 총 매출액을 집계한 뒤, 총 매출액이 **상위 10% 이상(>= 0.9 quantile)**인 날짜들을 **'High_Sales_Day'**로 정의합니다. 이 'High_Sales_Day'에 해당하는 날들의 일평균 매출액을 구하시오. (단, 소수점 이하는 버리고 **정수(int)**로 출력하시오.)

In [17]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n_rows = 5000

# 1. 상품명 생성 (대괄호 안에 옵션 포함)
products_base = ['Laptop', 'Phone', 'Monitor', 'Mouse', 'Keyboard']
options = ['[Black]', '[White]', '[Silver]', '[16GB]', '[Wireless]']
categories = ['Electronics', 'Electronics', 'Electronics', 'Accessories', 'Accessories']

prod_list = []
cat_list = []
for _ in range(n_rows):
    idx = np.random.randint(0, 5)
    opt_idx = np.random.randint(0, 5)
    # 상품명 예: "Laptop [Black]"
    prod_name = f"{products_base[idx]} {options[opt_idx]}"
    prod_list.append(prod_name)
    cat_list.append(categories[idx])

# 2. 데이터 생성
df = pd.DataFrame({
    'Date': np.random.choice(pd.date_range('2023-01-01', '2023-12-31'), n_rows),
    'Product': prod_list,
    'Category': cat_list,
    'Price': np.random.randint(50, 2000, n_rows) * 1000, # 50,000 ~ 2,000,000
    'Quantity': np.random.randint(1, 5, n_rows)
})

# 3. 결측치 생성 (Price)
df.loc[np.random.choice(df.index, 500), 'Price'] = np.nan

print(f"데이터 생성 완료! (행: {df.shape[0]})")
print(df.head())

데이터 생성 완료! (행: 5000)
        Date             Product     Category      Price  Quantity
0 2023-08-11  Monitor [Wireless]  Electronics  1030000.0         2
1 2023-12-26       Laptop [16GB]  Electronics        NaN         1
2 2023-10-19    Mouse [Wireless]  Accessories   410000.0         4
3 2023-07-24      Laptop [Black]  Electronics   930000.0         1
4 2023-03-09     Monitor [White]  Electronics  1437000.0         4


In [18]:
# 문제 1
df['Option'] = df['Product'].str.extract(r'\[(.*?)\]')
result = df['Option'].value_counts().sort_values(ascending=False)
print(result.idxmax())

Black


In [19]:
# 문제 2
fill_val = df.groupby('Category')['Price'].transform('mean')

df['Price'] = df['Price'].fillna(fill_val)
result = df['Price'].std()
print(int(result))

533145


In [20]:
# 문제 3
df['Revenue'] = df['Price'] * df['Quantity']
daily_sales = df.groupby('Date')['Revenue'].sum()
#print(daily_sales)

var = daily_sales.quantile(0.9)
high_sales_days = daily_sales[daily_sales >= var]
result = high_sales_days.mean()
print(int(result))

55587015


📝 실전 연습 문제 (데이터셋: Ecommerce_Sales)
문제 1. (문자열 포함 여부 검색) Product 이름에 **'Wireless'**라는 단어가 포함된 상품들만 별도로 분석하려고 합니다. 'Wireless'가 포함된 상품들의 **Price 중앙값(median)**을 구하시오. (단, Price 컬럼의 결측치는 제거(dropna)한 후 계산하고, 결과는 **정수(int)**로 출력하시오.)

문제 2. (시계열 집계 & 정렬) Date 컬럼에서 '월(Month)' 정보를 추출합니다. 'Electronics' 카테고리에 해당하는 데이터만 필터링한 후, **월별 총 매출액(Price * Quantity)**을 집계했을 때, 매출액이 가장 높은 **월(Month)**은 몇 월입니까? (단, 결측치가 있는 행은 매출액 계산 전 제거하고, 정수로 출력하시오.)

문제 3. (순위 구하기 & 인덱싱) 전체 데이터(결측치 제거 후)를 Price(가격)가 높은 순서대로 내림차순 정렬합니다. 이때, 가격이 5번째로 높은 주문의 **Order_ID**는 무엇입니까? (주의: 0부터 시작하는 인덱스 기준이 아니라, 실제 순위 기준 5등입니다. 동점자는 무시하고 행 순서대로 처리합니다.)

In [21]:
print(df.head())

        Date             Product     Category         Price  Quantity  \
0 2023-08-11  Monitor [Wireless]  Electronics  1.030000e+06         2   
1 2023-12-26       Laptop [16GB]  Electronics  1.022305e+06         1   
2 2023-10-19    Mouse [Wireless]  Accessories  4.100000e+05         4   
3 2023-07-24      Laptop [Black]  Electronics  9.300000e+05         1   
4 2023-03-09     Monitor [White]  Electronics  1.437000e+06         4   

     Option       Revenue  
0  Wireless  2.060000e+06  
1      16GB  1.022305e+06  
2  Wireless  1.640000e+06  
3     Black  9.300000e+05  
4     White  5.748000e+06  


In [22]:
# 문제 1
q1 = df.dropna(subset=['Price'])
result = q1[q1['Product'].str.contains('Wireless')]['Price'].median()
print(int(result))

1022305


In [23]:
# 문제 2
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month

q1 = df[df['Category']=='Electronics']
q1['Revenue'] = q1['Price'] * q1['Quantity']
result = q1.groupby('Month')['Revenue'].sum().sort_values(ascending=False)
print(result.idxmax())

9


C:\Users\sangh\AppData\Local\Temp\ipykernel_4896\3975182502.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1['Revenue'] = q1['Price'] * q1['Quantity']


In [24]:
# 문제 3
df = df.dropna(subset=['Price'])
df = df.sort_values(by='Price', ascending=False)
print(df.iloc[4])

Date        2023-11-20 00:00:00
Product      Monitor [Wireless]
Category            Electronics
Price                 1998000.0
Quantity                      3
Option                 Wireless
Revenue               5994000.0
Month                        11
Name: 2770, dtype: object


📝 [제2유형] 은행 고객 이탈 예측
문제: 은행 고객 데이터(train)를 이용하여 고객의 이탈 여부(Exited)를 예측하는 모델을 만들고, 제공된 평가용 데이터(test)에 대한 예측 결과를 csv 파일로 제출하시오.

1. 데이터 설명

Target: Exited (1: 이탈, 0: 유지)

Features:

CustomerId: 고객 ID (식별자)

Surname: 성 (String)

CreditScore: 신용 점수

Geography: 국가 (France, Germany, Spain)

Gender: 성별

Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary

2. 제출 형식

result.csv 파일로 저장해야 합니다.

반드시 **CustomerId**와 Exited 두 개의 컬럼만 포함해야 합니다.

Exited 컬럼에는 **이탈할 확률(Probability)**을 제출하시오. (0과 1 사이의 소수)

3. 평가 지표

ROC-AUC (값이 클수록 좋음)

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(2025)
n_rows = 3000

# 1. 데이터 생성
data = {
    'CustomerId': np.arange(10001, 10001 + n_rows),
    'Surname': np.random.choice(['Smith', 'Johnson', 'Williams', 'Brown', 'Jones'], n_rows),
    'CreditScore': np.random.randint(350, 850, n_rows),
    'Geography': np.random.choice(['France', 'Germany', 'Spain'], n_rows),
    'Gender': np.random.choice(['Male', 'Female'], n_rows),
    'Age': np.random.randint(18, 92, n_rows),
    'Tenure': np.random.randint(0, 10, n_rows),
    'Balance': np.random.uniform(0, 250000, n_rows),
    'NumOfProducts': np.random.randint(1, 4, n_rows),
    'HasCrCard': np.random.randint(0, 2, n_rows),
    'IsActiveMember': np.random.randint(0, 2, n_rows),
    'EstimatedSalary': np.random.uniform(11, 200000, n_rows),
    'Exited': np.random.randint(0, 2, n_rows) # Target
}

df = pd.DataFrame(data)

# 2. 결측치 및 이상치 강제 주입 (전처리 연습용)
# Age 결측치
df.loc[np.random.choice(df.index, 50), 'Age'] = np.nan
# Balance 결측치
df.loc[np.random.choice(df.index, 30), 'Balance'] = np.nan

# 3. Train / Test 분리 (시험 환경 흉내)
X = df.drop('Exited', axis=1)
y = df['Exited']

# 실제 시험에서는 X_test에 정답(y)이 없습니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 데이터프레임 합치기 (시험에서 제공되는 형태)
train = pd.concat([X_train, y_train], axis=1)
test = X_test # y_test는 숨겨져 있음 (채점용)

# CSV 파일로 저장되었다고 가정하고 변수명만 사용합니다.
print(f"데이터 생성 완료!")
print(f"train shape: {train.shape}")
print(f"test shape: {test.shape}")
print(df.head())

데이터 생성 완료!
train shape: (2100, 13)
test shape: (900, 12)
   CustomerId   Surname  CreditScore Geography  Gender   Age  Tenure  \
0       10001  Williams          423    France    Male  43.0       5   
1       10002     Jones          532   Germany  Female  54.0       9   
2       10003     Smith          470     Spain    Male  75.0       8   
3       10004     Brown          627   Germany  Female  81.0       0   
4       10005     Brown          520     Spain    Male  67.0       5   

         Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0  156207.782854              3          0               0     61837.658280   
1  149914.101471              2          1               1     54490.748664   
2  132768.028528              3          0               1    126846.371424   
3   44238.586796              1          0               0    172944.034041   
4  105158.756047              2          1               1     58251.970535   

   Exited  
0       0  
1       0  

In [26]:
#print(train.info())
# 결측치: Age, Balance
# object: Geography, Gender

#print(test.info())
# 결측치: Age, Balance

train['Age'] = train['Age'].fillna(train['Age'].mean())
test['Age'] = test['Age'].fillna(train['Age'].mean())

train['Balance'] = train['Balance'].fillna(train['Balance'].mean())
test['Balance'] = test['Balance'].fillna(train['Balance'].mean())

#print(test.info())

X = train.drop(['CustomerId', 'Surname', 'Exited'], axis=1)
y = train['Exited']
X_submit = test.drop(['CustomerId', 'Surname'], axis=1)

cols = ['Geography', 'Gender']

X = pd.get_dummies(X, columns=cols)
X_submit = pd.get_dummies(X_submit, columns=cols)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)
pred_proba = model.predict_proba(X_val)[:, 1]

from sklearn.metrics import roc_auc_score
roc = roc_auc_score(y_val, pred_proba)
print(round(roc, 3))

model.fit(X, y)
final_pred = model.predict(X_submit)

result = pd.DataFrame({
    'Id': test['CustomerId'],
    'pred': final_pred
})
result.to_csv('result.csv', index=False)
print(pd.read_csv('result.csv').head())



0.532
      Id  pred
0  11802     1
1  11191     1
2  11818     1
3  10252     1
4  12506     1


📝 [제2유형] 자전거 대여량 예측 (회귀)
문제: 제공된 자전거 대여 데이터(train)를 이용하여 대여 횟수(count)를 예측하는 모델을 만들고, 평가용 데이터(test)에 대한 예측 결과를 csv 파일로 제출하시오.

1. 데이터 설명

Target: count (대여 횟수, 수치형)

Features:

ID: 고유 식별자

Date: 날짜 (datetime)

Temperature, Humidity, Windspeed, Visibility: 날씨 정보

Holiday: 공휴일 여부 (Yes/No)

Hour: 시간 (0~23)

2. 제출 형식

result.csv 파일로 저장해야 합니다.

**ID**와 pred 두 개의 컬럼만 존재해야 합니다.

pred 컬럼에는 예측한 대여 횟수(실수 또는 정수)를 입력하시오.

3. 평가 지표

MAE (Mean Absolute Error)

(값이 작을수록 좋은 성능입니다.)

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(2025)
n_rows = 2000

# 1. 데이터 생성
data = {
    'ID': np.arange(1000, 1000 + n_rows),
    'Date': pd.date_range('2023-01-01', periods=n_rows, freq='h'),
    'Temperature': np.random.normal(20, 5, n_rows), # 기온
    'Humidity': np.random.randint(30, 90, n_rows),  # 습도
    'Windspeed': np.random.uniform(0, 10, n_rows),  # 풍속
    'Visibility': np.random.randint(500, 2000, n_rows), # 가시거리
    'Holiday': np.random.choice(['No', 'Yes'], n_rows, p=[0.95, 0.05]), # 공휴일
    'Hour': np.random.randint(0, 24, n_rows), # 시간
    'count': 0 # Target 초기화
}

df = pd.DataFrame(data)

# Target 생성 (대여량은 기온이 높고, 퇴근시간에 많다는 가정)
df['count'] = 100 + (df['Temperature'] * 3) + (df['Hour'] * 5) - (df['Humidity'] * 0.5)
df['count'] = df['count'] + np.random.normal(0, 20, n_rows) # 노이즈 추가
df['count'] = df['count'].apply(lambda x: max(0, int(x))) # 음수 방지 및 정수화

# 2. 결측치 주입 (전처리 연습용)
df.loc[np.random.choice(df.index, 50), 'Windspeed'] = np.nan

# 3. Train / Test 분리
X = df.drop('count', axis=1)
y = df['count']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

train = pd.concat([X_train, y_train], axis=1)
test = X_test

print(f"데이터 생성 완료!")
print(f"train shape: {train.shape}")
print(f"test shape: {test.shape}")
print(train.head())

데이터 생성 완료!
train shape: (1400, 9)
test shape: (600, 8)
        ID                Date  Temperature  Humidity  Windspeed  Visibility  \
836   1836 2023-02-04 20:00:00    27.741598        39   4.947782         836   
575   1575 2023-01-24 23:00:00    20.760819        80        NaN        1787   
557   1557 2023-01-24 05:00:00    15.534825        61   2.401177        1261   
1235  2235 2023-02-21 11:00:00    26.003205        64   7.495187         971   
1360  2360 2023-02-26 16:00:00    18.989828        52   0.973485         958   

     Holiday  Hour  count  
836       No    14    205  
575       No     5    153  
557       No    10    204  
1235      No     7    152  
1360      No     8    138  


In [28]:
#print(test.info())
# 결측치: windspeed
# 인코딩: Holiday
# 삭제: ID, Date

train['Windspeed'] = train['Windspeed'].fillna(train['Windspeed'].mean())
test['Windspeed'] = test['Windspeed'].fillna(train['Windspeed'].mean())

X = train.drop(['ID', 'Date', 'count'], axis=1)
y = train['count']
X_submit = test.drop(['ID', 'Date'], axis=1)

X = pd.get_dummies(X, columns=['Holiday'])
X_submit = pd.get_dummies(X_submit, columns=['Holiday'])

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_val)

from sklearn.metrics import mean_absolute_error
pred_val = mean_absolute_error(y_val, pred)
print(round(pred_val, 3))

model.fit(X, y)
final_pred = model.predict(X_submit)

result = pd.DataFrame({
    'ID': test['ID'],
    'pred': final_pred
})
result.to_csv('result1.csv', index=False)
print(pd.read_csv('result1.csv').head())

17.27
     ID    pred
0  2860  127.66
1  1353  118.44
2  2333  182.31
3  1905  221.30
4  2289  205.26


📝 [제2유형] 전자상거래 구매 예측
문제: 제공된 고객 데이터(train)를 이용하여 고객이 상품을 구매할지(Purchase) 여부를 예측하는 모델을 만들고, 평가용 데이터(test)에 대한 예측 결과를 result.csv 파일로 제출하시오.

1. 데이터 설명

Target: Purchase (1: 구매, 0: 비구매)

Features:

UserID: 고객 ID (불필요한 식별자)

Age, AnnualIncome, BrowsingTime, PagesViewed: 수치형 정보

MembershipTier: 회원 등급 (Bronze, Silver, Gold)

DeviceType: 접속 기기 (Mobile, Desktop, Tablet)

2. 제출 형식

result.csv 파일로 저장.

**UserID**와 pred 두 개의 컬럼만 포함.

pred 컬럼에는 **구매할 확률(Probability)**을 제출하시오. (0~1 사이 값)

3. 평가 지표

ROC-AUC

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(2025)
n_rows = 4000

# 1. 데이터 생성
data = {
    'UserID': np.arange(10001, 10001 + n_rows),
    'Age': np.random.randint(18, 70, n_rows),
    'AnnualIncome': np.random.randint(20000, 150000, n_rows),
    'BrowsingTime': np.random.normal(30, 10, n_rows), # 분 단위
    'PagesViewed': np.random.randint(1, 20, n_rows),
    'MembershipTier': np.random.choice(['Bronze', 'Silver', 'Gold'], n_rows, p=[0.5, 0.3, 0.2]),
    'DeviceType': np.random.choice(['Mobile', 'Desktop', 'Tablet'], n_rows),
    'Purchase': 0 # Target 초기화
}

df = pd.DataFrame(data)

# 2. Target 생성 (구매 확률: 골드 회원일수록, 수입이 많을수록, 많이 볼수록 높음)
logits = -5 + (df['AnnualIncome'] / 50000) + (df['BrowsingTime'] / 20) + (df['PagesViewed'] / 5)
logits += np.where(df['MembershipTier'] == 'Gold', 2, 0)
logits += np.where(df['MembershipTier'] == 'Silver', 1, 0)
probs = 1 / (1 + np.exp(-logits))
df['Purchase'] = np.random.binomial(1, probs)

# 3. 데이터 분리
X = df.drop('Purchase', axis=1)
y = df['Purchase']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 시험 제공 형태
train = pd.concat([X_train, y_train], axis=1)
test = X_test

print("데이터 생성 완료! (결측치 없음)")
print(f"train shape: {train.shape}")
print(f"test shape: {test.shape}")

데이터 생성 완료! (결측치 없음)
train shape: (2800, 8)
test shape: (1200, 7)


In [62]:
#print(test.info())
# 제거: UserID
# 인코딩: MembershipTier, DeviceType
# 확률 -> classfier

X = train.drop(['UserID', 'Purchase'], axis=1)
y = train['Purchase']
X_submit = test.drop('UserID', axis=1)

cols = ['MembershipTier', 'DeviceType']

X = pd.get_dummies(X, columns=cols)
X_submit = pd.get_dummies(X_submit, columns=cols)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
pred_proba = model.predict_proba(X_val)[:,1]

from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pred_proba)
print(round(score, 3))

model.fit(X, y)
pred_proba_final = model.predict_proba(X_submit)[:,1]

result = pd.DataFrame({
    'UserID': test['UserID'],
    'pred': pred_proba_final
})

result.to_csv('result2.csv', index=False)
print(pd.read_csv('result2.csv'))

0.787
      UserID  pred
0      10556  0.51
1      13492  0.71
2      10528  0.99
3      13926  0.62
4      12990  0.95
...      ...   ...
1195   13857  0.73
1196   10227  0.50
1197   11613  0.91
1198   10536  0.64
1199   13849  0.66

[1200 rows x 2 columns]


📝 [제3유형] 통계적 가설 검정
문제 1. (대응표본 t-검정) 'Treatment'(투약군) 그룹에 속한 환자들의 **투약 전(BP_Pre)**과 투약 후(BP_Post) 혈압 차이가 유의미한지 검정하려고 합니다. (가설: 투약 후 혈압이 투약 전과 다르다.) **대응표본 t-검정(Paired t-test)**을 수행하고, **검정통계량(t-value)**을 구하시오. (단, 반올림하여 소수 셋째 자리까지 출력하시오.)

문제 2. (카이제곱 독립성 검정) 투여 그룹(Group)과 부작용 발생 여부(Side_Effects)가 서로 관련이 있는지(독립이 아닌지) 검정하려고 합니다. 카이제곱 독립성 검정을 수행하고, **p-값(p-value)**을 구하시오. (단, 반올림하여 소수 넷째 자리까지 출력하시오.)

문제 3. (로지스틱 회귀분석 & 오즈비) 치료 성공 여부(Success)를 종속변수로 하고, Age와 BMI를 독립변수로 하는 로지스틱 회귀분석을 수행하시오. 이때 **BMI 변수의 오즈비(Odds Ratio)**를 구하시오. (단, statsmodels를 사용하고, 결과값은 반올림하여 소수 셋째 자리까지 출력하시오.)

In [29]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n = 300

data = {
    'ID': range(1, n + 1),
    'Group': np.random.choice(['Treatment', 'Control'], n),
    'Age': np.random.randint(25, 80, n),
    'BMI': np.random.normal(25, 4, n),
    'BP_Pre': np.random.normal(130, 10, n),
    'Side_Effects': np.random.choice(['Yes', 'No'], n, p=[0.15, 0.85])
}

df = pd.DataFrame(data)

# 투약 후 혈압 생성 (Treatment 그룹만 혈압이 떨어지는 효과 부여)
def get_bp_post(row):
    change = np.random.normal(-5, 5) if row['Group'] == 'Treatment' else np.random.normal(0, 3)
    return row['BP_Pre'] + change

df['BP_Post'] = df.apply(get_bp_post, axis=1)

# 치료 성공 여부 (BMI가 낮을수록, 나이가 적을수록 성공 확률 높음)
logits = 2 - 0.1 * df['BMI'] - 0.02 * df['Age'] + np.random.normal(0, 1, n)
probs = 1 / (1 + np.exp(-logits))
df['Success'] = np.random.binomial(1, probs)

print("데이터 생성 완료!")
print(df.head())

데이터 생성 완료!
   ID      Group  Age        BMI      BP_Pre Side_Effects     BP_Post  Success
0   1  Treatment   49  25.536214  115.545523          Yes  107.602568        0
1   2  Treatment   75  25.708310  118.339493           No  114.446628        0
2   3  Treatment   45  25.510836  115.130393           No  114.998380        0
3   4  Treatment   29  27.439367  128.171946           No  125.530657        0
4   5  Treatment   32  19.326160  129.338332           No  119.815970        0


In [30]:
# 문제 1
target_df = df[df['Group']=='Treatment']
before = target_df['BP_Pre']
after = target_df['BP_Post']

from scipy.stats import ttest_rel
t_stat, p_val = ttest_rel(before, after)
print(round(t_stat, 3))

11.205


In [31]:
# 문제 2
from scipy.stats import chi2_contingency
group = df['Group']
side_effects = df['Side_Effects']

ct = pd.crosstab(group, side_effects)

chi2, p_val, ddof, expected = chi2_contingency(ct)
print(round(p_val, 3))

0.283


In [32]:
# 문제 3
from statsmodels.formula.api import logit
model = logit('Success ~ Age + BMI', data=df).fit()
#print(model.summary())

import numpy as np
odds_ratio = np.exp(model.params['BMI'])
print(round(odds_ratio, 3))

Optimization terminated successfully.
         Current function value: 0.465692
         Iterations 6
0.939


📝 [제3유형] 회귀분석 종합 문제
문제 1. (다중회귀분석 - OLS) 투약 후 혈압(BP_Post)을 종속변수로 하고, 투약 전 혈압(BP_Pre), 나이(Age), 체질량지수(BMI)를 독립변수로 하는 다중선형회귀 모델을 생성하시오. 이때, **BP_Pre (투약 전 혈압)의 회귀계수(Coefficient)**를 구하시오. (단, 소수점 넷째 자리에서 반올림하여 셋째 자리까지 출력하시오.)

문제 2. (로지스틱 회귀 - 범주형 변수의 오즈비) 치료 성공 여부(Success)를 종속변수로 하고, Age, BMI, Group을 독립변수로 하는 로지스틱 회귀 모델을 생성하시오. 이때, 'Treatment'(투약군) 그룹의 환자는 'Control'(대조군) 그룹의 환자보다 치료 성공 오즈(Odds)가 몇 배인지(오즈비) 구하시오. (단, Control을 기준(0), Treatment를 비교(1)로 처리하며, 오즈비는 소수점 넷째 자리에서 반올림하여 셋째 자리까지 출력하시오.)

문제 3. (로지스틱 회귀 - 확률 예측) 2번 문제에서 만든 로지스틱 회귀 모델을 사용하여, 전체 데이터(df) 환자들의 **치료 성공 확률(Probability)**을 예측하시오. 예측된 치료 성공 확률이 **0.6 이상(>= 0.6)**인 환자의 **인원수(명)**를 구하시오.

In [33]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n = 500

# 데이터 생성
data = {
    'ID': range(1, n + 1),
    'Group': np.random.choice(['Treatment', 'Control'], n), # 투약군, 대조군
    'Age': np.random.randint(25, 80, n),
    'BMI': np.random.normal(25, 4, n),
    'BP_Pre': np.random.normal(130, 10, n), # 투약 전 혈압
}

df = pd.DataFrame(data)

# 종속변수 생성 (선형 관계)
# 투약 후 혈압 = 투약 전 * 0.9 + 나이 * 0.1 + BMI * 0.5 - (Treatment인 경우 10)
def get_bp_post(row):
    effect = -10 if row['Group'] == 'Treatment' else 0
    base = row['BP_Pre'] * 0.9 + row['Age'] * 0.1 + row['BMI'] * 0.5 + 10
    return base + effect + np.random.normal(0, 2, 1)[0]

df['BP_Post'] = df.apply(get_bp_post, axis=1)

# 종속변수 생성 (로지스틱 관계)
# Treatment일 때 성공 확률 증가, BMI 낮을수록 성공
df['Treatment_Code'] = df['Group'].map({'Treatment': 1, 'Control': 0})
logits = -2 + 1.5 * df['Treatment_Code'] - 0.1 * df['BMI'] + 0.02 * df['Age']
probs = 1 / (1 + np.exp(-logits))
df['Success'] = np.random.binomial(1, probs)

print("데이터 생성 완료!")
print(df.head(3))

데이터 생성 완료!
   ID      Group  Age        BMI      BP_Pre     BP_Post  Treatment_Code  \
0   1  Treatment   39  30.087421  128.659704  135.787762               1   
1   2  Treatment   75  27.065314  128.106733  136.317723               1   
2   3  Treatment   73  25.347528  138.199694  144.206251               1   

   Success  
0        0  
1        1  
2        1  


In [ ]:
# 1번 문제
from statsmodels.formula.api import ols
model = ols('BP_Post ~ BP_Pre + Age + BMI', data=df).fit()
coef = model.params['BP_Pre']
print(round(coef, 3))

0.895


In [ ]:
# 2번 문제
from statsmodels.formula.api import logit
model = logit('Success ~ Age + BMI + Group', data=df).fit()
coef = model.params['Group[T.Treatment]']
# 설명 (자동 더미 변수화 - 문자열)
odds_ratio = np.exp(coef)
print(round(odds_ratio, 3))

Optimization terminated successfully.
         Current function value: 0.208368
         Iterations 9
19.577


In [ ]:
# 3번 문제 - 외우기
pred_proba = model.predict(df)
target = pred_proba[pred_proba >= 0.6]
result = len(target)
print(result)

0


📝 [제3유형] 분산분석표 (ANOVA Table) 해석
문제 1. (ANOVA 모델 적합) 식이요법 유형(Diet)에 따른 체중 감량(Weight_Loss)의 차이를 분석하기 위해, ols 함수를 사용하여 선형 회귀 모델을 생성하고 적합(fit)시키시오. 생성된 모델 객체를 이용하여 anova_lm 함수를 실행하고, 결과 테이블에서 **Diet 변수의 F-통계량(F-value)**을 구하시오. (단, 반올림하여 소수 셋째 자리까지 출력하시오.)

문제 2. (잔차 제곱합 - SSE) 1번에서 구한 분산분석표(ANOVA Table)를 확인하여, 설명되지 않는 변동인 **잔차(Residual)의 제곱합(Sum of Squares)**을 구하시오. (단, 반올림하여 소수 셋째 자리까지 출력하시오.)

문제 3. (p-value 해석) 분산분석 결과의 **p-값(PR(>F))**을 구하고, 유의수준 0.05 하에서 귀무가설(세 그룹의 평균은 같다)을 기각할 수 있는지 판단하시오. (p-값은 소수 넷째 자리까지 반올림하여 출력하고, 기각 여부는 "기각" 또는 "채택"으로 출력하시오. 정답 예시: 0.0123 기각)

In [39]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n = 150

# 데이터 생성
data = {
    'ID': range(1, n + 1),
    'Diet': np.random.choice(['Type_A', 'Type_B', 'Type_C'], n),
    'Weight_Loss': np.zeros(n)
}

df = pd.DataFrame(data)

# 그룹별 효과 부여 (C > B > A 순으로 감량 효과 큼)
def get_weight_loss(diet):
    base = np.random.normal(2, 1) # 기본 감량
    if diet == 'Type_B': return base + 2  # B는 +2kg 더 감량
    elif diet == 'Type_C': return base + 4 # C는 +4kg 더 감량
    else: return base # A는 기본

df['Weight_Loss'] = df['Diet'].apply(get_weight_loss)

print("데이터 생성 완료!")
print(df.head())

데이터 생성 완료!
   ID    Diet  Weight_Loss
0   1  Type_C     6.616633
1   2  Type_C     5.644096
2   3  Type_C     6.007296
3   4  Type_C     3.862329
4   5  Type_A     2.226040


In [44]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
model = ols('Weight_Loss ~C(Diet)', data=df).fit()
anova_table = anova_lm(model)
print(anova_table)

f_stat = anova_table.loc['C(Diet)', 'F']
print(round(f_stat, 3))

sse = anova_table.loc['Residual', 'sum_sq']
print(round(sse, 3))

p_val = anova_table.loc['C(Diet)', 'PR(>F)']
print(round(p_val, 4))

             df      sum_sq     mean_sq          F        PR(>F)
C(Diet)     2.0  433.580080  216.790040  238.35233  7.359841e-47
Residual  147.0  133.701801    0.909536        NaN           NaN
238.352
133.702
0.0
